Implementation of Chatbot using NLP

In [1]:
pip install nltk scikit-learn streamlit


  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 325.1 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 325.1 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 230.7 kB/s eta 0:00:00
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.me


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [4]:
intents = [
    {
        'tag':'greeting',
        'patterns':['Hi', 'Hello', 'Hey', 'Whats up', 'How are you'],
        'responses':['Hi there', 'Hello', 'Hey', 'Nothing much', 'Im fine, thank you']
    },
    {
        'tag':'goodbye',
        'patterns':['Bye', 'See you later', 'Goodbye', 'Take care'],
        'responses':['Goodbye', 'See you later', 'Take care']
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
    "tag": "jokes",
    "patterns": ["Tell me a joke", "Make me laugh", "Say something funny"],
    "responses": ["Why don’t scientists trust atoms? Because they make up everything!","Why did the scarecrow win an award? Because he was outstanding in his field!","I told my wife she should embrace her mistakes. She gave me a hug."]
    },
    {
    "tag": "motivation",
    "patterns": ["Motivate me", "I need inspiration", "Tell me something inspiring"],
    "responses": ["Believe in yourself! You are capable of achieving great things.", "Every day is a new beginning. Stay positive and keep pushing forward!","Success is not final, failure is not fatal: It is the courage to continue that counts."]
    },
    {
    "tag": "time",
    "patterns": ["What time is it", "Tell me the time", "Can you give me the current time?"],
    "responses": ["I'm not a clock, but you can check the time on your device.","Time is just an illusion! But you can check your phone for the exact time.","I'm unable to check the time, but it's always the right time to be productive!"]
    },
    {
    "tag": "date",
    "patterns": ["What is today's date", "Tell me the date", "What day is it today"],
    "responses": ["I'm not a calendar, but you can check your device for the date.","Every day is a fresh start! But if you need the exact date, check your phone.","I'm not sure, but I can tell you it's a great day to do something amazing!"]
    },
    {
    "tag": "compliment",
    "patterns": ["Say something nice", "Give me a compliment", "Make me feel good"],
    "responses": ["You're amazing just the way you are!","You're smart, talented, and capable of achieving anything!","You have a great personality, and the world is lucky to have you!"]
    },
    {
    "tag": "food",
    "patterns": ["What should I eat today", "Suggest me a meal", "I'm hungry"],
    "responses": ["How about some pasta with a side of garlic bread?","A classic burger with fries always hits the spot!","Try something healthy like a bowl of salad with grilled chicken."]
    },
    {
    "tag": "coding_help",
    "patterns": ["How do I debug my code", "Help me with programming", "I need coding assistance"],
    "responses": ["Try checking the error message and searching online for solutions.","Break your code into smaller parts and test each one separately.","Let me know what error you're facing, and I'll try to help!"]
    }
]

In [2]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

#Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

NameError: name 'TfidfVectorizer' is not defined

In [3]:
# Training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

NameError: name 'vectorizer' is not defined

In [4]:
# Python function to chat with the chatbot
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [5]:
# Checking our chatbot.
user_input = "What's your age?"
response = chatbot(user_input)
print(response)

NameError: name 'vectorizer' is not defined

In [6]:
user_input = "Hello"
response = chatbot(user_input)
print(response)

NameError: name 'vectorizer' is not defined

In [7]:
user_input = "budget"
response = chatbot(user_input)
print(response)

NameError: name 'vectorizer' is not defined

In [8]:
user_input = "hi"
response = chatbot(user_input)
print(response)

NameError: name 'vectorizer' is not defined

In [9]:
user_input = "age"
response = chatbot(user_input)
print(response)

NameError: name 'vectorizer' is not defined